In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import selenium
print(selenium.__version__)

4.8.2


# 4Chan

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
# browser.set_window_position(1920,0)
browser.get("https://boards.4channel.org/g/thread/92468569#p92471725")
# href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
# href = href.split("&url=")[1]
# browser.get(href)
# browser.close()
# del browser
# print(f"href: {href}")

In [17]:
selector = browser.execute_script(r"return {a:4, b:6, myName:'is daddy'};")
print(selector) # holy shit it converts the objects

{'a': 4, 'b': 6, 'myName': 'is daddy'}


In [ ]:
import requests
url = "http://127.0.0.1:5000/"
#     prompt = f"""The following is a conversation between a human and an AI.
# Human: {human_input}
# AI:"""
prompt=f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that correctly completes the request.

### Instruction:
{human_input}

### Context:
{human_input}

### Response:"""
data = {"prompts": [prompt]}
response = requests.post(url, json=data).json()[0]
response = response.replace(prompt, "")
if "Human:" in response:
    response = response.split("Human:")[0]
response = response.strip()
return response
# await get_response("Hi there!")

# Picker

In [2]:
import json

class AutoSaveDict(dict):
    def __init__(self, json_file):
        self.json_file = json_file
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
                super().__init__(data)
        except FileNotFoundError:
            super().__init__()

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.save_to_json()

    def save_to_json(self):
        with open(self.json_file, 'w') as f:
            json.dump(self, f, indent=4)
config = AutoSaveDict("config.json")

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Edge()
driver.get("https://twitter.com")

In [22]:
print(driver.get_window_rect())

{'height': 1020, 'width': 945, 'x': 1983, 'y': 14}


In [82]:
async def getSelector(thing: str):
    print("Prompting user to click")
    selector = driver.execute_async_script(open("picker.js","r").read())
    print(f"Found element with selector {selector}")
    selector = selector.replace("./div/", "")
    print(f"Shortened selector: {selector}")

    import requests
    url = "http://127.0.0.1:5000/"
    PROMPT_DICT = {
        "prompt_input": (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that correctly completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
        ),
        "prompt_no_input": (
            "Below is an instruction that describes a task. "
            "Write a response that correctly completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n"
        ),
    }
    def fmt(prompt: str, input: str = "") -> str:
        if input.strip() == "":
            return PROMPT_DICT['prompt_no_input'].format_map({'instruction': prompt})
        else:
            return PROMPT_DICT['prompt_input'].format_map({'instruction': prompt, 'input': input})
        
    prompts=[
    # fmt(
    #     """You are a calibration engine for a web interface.
    # The user has just indicated the log in button on a webpage.
    # Clean up the CSS selector for the log in button.
    # Remove any unnecessary parts, and make sure it is as short as possible.
    # You do not need to start at the root element.
    # Try and make it as short as possible.

    # ### Example Input:
    # body > div#react-root > div > div:n-th-of-type(2) > div > a[href="/home"] > div

    # ### Example Output:
    # a[href="/home"]
    # """,

    #     result),

#         fmt(
#         """You are a calibration engine for a web interface.
# The user has just indicated the {thing} on a webpage.
# Clean up the XPath selector for the {thing}.
# Remove any unnecessary parts, and make sure it is as short as possible.

# ### Example Input:
# /html/body/div[@id="react-root"]/div/div[2]/div/a[@href="/home"]/div

# ### Example Output:
# //a[@href="/home"]

# ### Example Input:
# /./body/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./span[contains(., 'Log out')]/./span[contains(., 'Log out')]

# ### Example Output:
# //span[text()[contains(., 'Log out')]]

# ### Example Input:
# /./div/./input[@autocomplete='username']/./input[@autocomplete='username']

# ### Example Output:
# //input[@autocomplete='username']
# """,

fmt(f"""You are a calibration engine for a web interface.
The user has just indicated the {thing} element on a webpage.
Clean up the XPath selector for the {thing} element by removing the unimportant parts.
Try and make it as short as possible. Propose 3 different selectors separated by newlines.

### Example Input:
/html/body/div[@id="react-root"]/div/div[2]/div/a[@href="/home"]/div

### Example Output:
//a[@href="/home"]

### Example Input:
/./body/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./span[contains(., 'Log out')]/./span[contains(., 'Log out')]

### Example Output:
//span[text()[contains(., 'Log out')]]
""",

        selector),
    ]
    data = {"prompts": prompts}
    print("Refining selector with remote model")
    response = requests.post(url, json=data).json()
    # print(response)
    refinedSelector = response[0].split("### Response:")[1].split("###")[0].strip()
    print(f"Refined selector: {refinedSelector}")

    refinedSelectors = refinedSelector.split("\n")
    validSelectors = []
    for x in refinedSelectors:
        try:
            assert driver.find_element(By.XPATH, x) is not None
            validSelectors.append(x)
        except:
            pass
    refinedSelector = "\n".join(validSelectors)

    prompts = [
        fmt(f"""You are a calibration engine for a web interface.
Pick the XPath selector that best matches {thing} on a webpage.""",
        f"{selector}\n{refinedSelector}")
    ]
    data = {"prompts": prompts}
    response = requests.post(url, json=data).json()
    refinedSelector = response[0].split("### Response:")[1].split("###")[0].strip()
    print(f"Refined selector: {refinedSelector}")

    assert driver.find_element(By.XPATH, refinedSelector) is not None
    return refinedSelector
config["logInSubmitSelector"] = await getSelector("dialog login button, must be part of the form. There are more than one login buttons on the page, so be sure to be specific enough to only grab the one in the form")


Prompting user to click
Found element with selector /./body/./div[@id='react-root']/./div/./div/./div[@id='layers']/./div/./div/./div/./div[@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div/./div/./div[@data-testid='LoginForm_Footer_Container']/./div/./div/./div/./div[@data-testid='LoginForm_Login_Button'][@role='button'][@tabindex='0']/./div[@dir='ltr']/./span/./span
Shortened selector: /./body/./div[@id='react-root']/./div[@id='layers']/./div[@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@data-testid='LoginForm_Footer_Container']/./div[@data-testid='LoginForm_Login_Button'][@role='button'][@tabindex='0']/./div[@dir='ltr']/./span/./span
Refining selector with remote model
Refined selector: //span[text()[contains(., 'Login')

In [12]:
from time import sleep

In [13]:
def click(xpath):
    driver.execute_script(f"""
        document.evaluate(
            `{xpath}`, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null
        ).singleNodeValue.click();
    """)

In [14]:
def focus(xpath):
    """
    1. Insert a panel into the page to absorb a mouse click
    2. Click the browser content area to allow for programmatic focusing of elements (browser security thing)
    3. The panel removes itself
    4. The element is focused
    """
    import pyautogui
    driver.execute_script(r"""
const panel = document.createElement("div");
panel.style.position = "fixed";
panel.style.top = "0";
panel.style.left = "0";
panel.style.width = "100%";
panel.style.height = "100%";
panel.style.zIndex = "9999";
panel.style.background = "rgba(0,0.5,0,0.5)";
panel.innerText = "CLICK FOR FOCUS";
panel.style.color = "white";
panel.style.fontSize = "50px";
panel.style.fontFamily = "Impact, Charcoal, sans-serif";

panel.style.display = "flex";
panel.style.justifyContent = "center";
panel.style.alignItems = "center";
panel.style.cursor = "pointer";
document.body.appendChild(panel);
panel.addEventListener("click", () => {
    document.body.removeChild(panel);
});
""")
    # click on the browser window
    restore = pyautogui.position()
    pyautogui.click(driver.get_window_position()['x'] + driver.get_window_size()["width"]//4, driver.get_window_position()['y'] + driver.get_window_size()["height"]//2)
    pyautogui.moveTo(restore)

    driver.execute_script(f"""
        document.evaluate(
            `{xpath}`, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null
        ).singleNodeValue.focus();
    """)


In [15]:
config["logInSelector"] = await getSelector("log in button")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]
Refining selector with remote model
Refined selector: //span[text()[contains(., 'Log in')]]


In [16]:
click(config["logInSelector"])

In [17]:
config["usernameSelector"] = await getSelector("username input")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div[@id='layers']/./div/./div/./div/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./div/./div/./div/./input[@autocomplete='username']/./input[@autocomplete='username']/./input[@autocomplete='username']
Refining selector with remote model
Refined selector: //input[@autocomplete='username']


In [23]:
focus(config["usernameSelector"])

In [19]:
def didOnePassOpen():
    import mss
    import pyautogui
    from IPython.display import display
    from PIL import Image
    def getMouseMonitorScreenshot():
        # Get the current mouse position
        mouse_x, mouse_y = pyautogui.position()

        # Determine the monitor the mouse is on
        with mss.mss() as sct:
            target_monitor = None
            for monitor in sct.monitors[1:]:
                if monitor["left"] <= mouse_x < monitor["left"] + monitor["width"] and monitor["top"] <= mouse_y < monitor["top"] + monitor["height"]:
                    target_monitor = monitor
                    break

            if target_monitor:
                image = sct.grab(target_monitor)
                return Image.frombytes("RGB", image.size, image.bgra, "raw", "BGRX")
            else:
                return None
    import pytesseract
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    def recognize_text(image):
        return pytesseract.image_to_string(image, lang="eng")
    
    
    text = recognize_text(getMouseMonitorScreenshot())
    # print("OCR", text)
    if "Search 1Password" in text or "Q earch 1Password" in text:
        return True
    elif "Copy Username" in text and "Copy Password" in text:
        return True
    else:
        return False


In [70]:
import pyautogui
pyautogui.hotkey("ctrl","shift","space")
sleep(0.5)
if didOnePassOpen():
    pyautogui.typewrite("twitter")
    sleep(0.5)
    pyautogui.hotkey("ctrl","c")
    focus(config["usernameSelector"])
    sleep(0.5)
    pyautogui.hotkey("ctrl","a")
    pyautogui.hotkey("ctrl","v")
    

In [26]:
config["loginNextSelector"] = await getSelector("next button")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div[@id='layers']/./div/./div/./div/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./span[contains(., 'Next')]/./span[contains(., 'Next')]
Refining selector with remote model
Refined selector: //span[text()[contains(., 'Next')]]


In [71]:
click(config["loginNextSelector"])

In [53]:
config["passwordSelector"] = await getSelector("password input")

Prompting user to click
Found element with selector /./body/./div[@id='react-root']/./div/./div/./div[@id='layers']/./div/./div/./div/./div[@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div/./div[@data-viewportview='true']/./div/./div/./div/./div/./label[contains(., 'Password')]/./div/./div/./div[@dir='ltr']/./input[@autocomplete='current-password'][@autocapitalize='sentences'][@autocorrect='on'][@dir='auto'][@name='password'][@spellcheck='true'][@type='password'][@value='']
Shortened selector: /./body/./div[@id='react-root']/./div[@id='layers']/./div[@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@data-viewportview='true']/./label[contains(., 'Password')]/./div[@dir='ltr']/./input[@autocomplete='current-password'][@autocapit

In [54]:
focus(config["passwordSelector"])

In [72]:
pyautogui.hotkey("ctrl","shift","space")
sleep(0.5)
if didOnePassOpen():
    pyautogui.typewrite("twitter")
    sleep(0.5)
    pyautogui.hotkey("ctrl","shift","c")
    focus(config["passwordSelector"])
    sleep(0.5)
    pyautogui.hotkey("ctrl","a")
    pyautogui.hotkey("ctrl","v")
else:
    raise Exception("1Password not open")

In [63]:
config["logInSubmitSelector"] = await getSelector("dialog login button, must be part of the form. There are more than one login buttons on the page, so be sure to be specific enough to only grab the one in the form")

Prompting user to click
Found element with selector /./body/./div[@id='react-root']/./div/./div/./div[@id='layers']/./div/./div/./div/./div[@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div/./div/./div[@role='group'][@tabindex='0']/./div/./div/./div[@data-testid='LoginForm_Footer_Container']/./div/./div/./div/./div[@data-testid='LoginForm_Login_Button'][@role='button'][@tabindex='0']/./div[@dir='ltr']/./span/./span
Shortened selector: /./body/./div[@id='react-root']/./div[@id='layers']/./div[@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@aria-labelledby='modal-header'][@aria-modal='true'][@role='dialog']/./div[@role='group'][@tabindex='0']/./div[@data-testid='LoginForm_Footer_Container']/./div[@data-testid='LoginForm_Login_Button'][@role='button'][@tabindex='0']/./div[@dir='ltr']/./span/./span
Refining selector with remote model
Refined selector: //span[text()[contains(., 'LoginFo

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[text()[contains(., 'LoginForm_Footer_Container')]]
//span[text()[contains(., 'LoginForm_Login_Button')]]
//button[tabindex='0'][dir='ltr'][data-testid='LoginForm_Login_Button']"}
  (Session info: MicrosoftEdge=111.0.1661.62)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF72CB63C22+14194]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF72CAFAEC2+817458]
	(No symbol) [0x00007FF72C78DB50]
	(No symbol) [0x00007FF72C7D0585]
	(No symbol) [0x00007FF72C7D070C]
	(No symbol) [0x00007FF72C80C277]
	(No symbol) [0x00007FF72C7F06CF]
	(No symbol) [0x00007FF72C7C3121]
	(No symbol) [0x00007FF72C8096FE]
	(No symbol) [0x00007FF72C7F0343]
	(No symbol) [0x00007FF72C7C1FFC]
	(No symbol) [0x00007FF72C7C0F7E]
	(No symbol) [0x00007FF72C7C27EE]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF72C9DEAA0+1

In [94]:
click(config["logInSubmitSelector"])

# Tweet gatherer

In [ ]:
program = """
Go to twitter.com.
Check if the user is logged in. If there is """

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Edge()
driver.get("https://twitter.com")


# Forge MDK

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
browser.set_window_position(1920,0)
browser.get("https://files.minecraftforge.net/")
href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
href = href.split("&url=")[1]
browser.get(href)
# browser.close()
# del browser
print(f"href: {href}")

href: https://maven.minecraftforge.net/net/minecraftforge/forge/1.19.3-44.1.23/forge-1.19.3-44.1.23-mdk.zip


In [ ]:
%cd ../../src
from tools.forge_mdk import ForgeMDKTool
from tools.helpers import handle_tool_invocation
# tool = ForgeMDKTool()
# tool.invoke("latest")
await handle_tool_invocation("[EDGE(open https://files.minecraftforge.net/) ->")

d:\Repos\Other\toolformer\backend\src
hehe


Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packages\selenium\webdriver\common\service.py", line 72, in start
    self.process = subprocess.Popen(cmd, env=self.env,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 1440, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\src\tools\helpers.py", line 35, in handle_tool_invocation
    result = tool.method(params)
  File "d:\Repos\Other\toolformer\backend\src\tools\edge.py", line 31, in invoke
    driver = Edge(options=options)
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packa

(True,
 "ERROR: Message: 'msedgedriver' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687\n")